In [1]:
import numpy as np
import pandas as pd
import requests
import sqlite3
import os

In [13]:
##get bulk card locations and update times
files = requests.get('https://api.scryfall.com/bulk-data').json()

##get file names
files_dict = dict({})
for i in range(len(files['data'])):
    files_dict[files['data'][i]['type']] = i
files_dict

{'oracle_cards': 0,
 'unique_artwork': 1,
 'default_cards': 2,
 'all_cards': 3,
 'rulings': 4}

In [21]:
##set files to copy as to sql tables
files_to_copy = ['oracle_cards','all_cards']

In [22]:
##make or update scryfall files manifest
manifest_cols = ['scryfall_bulk_data_index','scryfall_data_desc','scryfall_updated_date']
manifest = pd.DataFrame(columns=manifest_cols)
for d in files_to_copy:
    df = pd.DataFrame([[files_dict[d],d,files['data'][files_dict[d]]['updated_at']]],columns=manifest_cols)
    manifest = pd.concat([manifest,df],ignore_index=True)

manifest.to_csv('scryfall_manifest.csv',index=False)

In [28]:
##open sql conn
db_path = os.path.join('database.db')
conn = sqlite3.connect(db_path)

##build the tables
for d in files_to_copy:
    url = files['data'][files_dict[d]]['download_uri']
    data = requests.get(url).json()
    columns = [c for c in data[0]]
    df = pd.DataFrame(data, columns=columns)
    for col in columns:
        df[col] = str(df[col])
    df.to_sql(name=d,con=conn, if_exists='replace', index=False)
    print('Full rebuild of',d,'complete.')


Full rebuild: oracle_cards done.
Full rebuild: all_cards done.


In [29]:
conn.commit()
conn.close()